# ANSWERS TO QUESTION ABOUT CHAPTER 4

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
LetsPlot.setup_html()# necessary to show plots on jupyter notebook

## QUESTION 2: APPLYING FWL THEOREM TO CALCULATE ATE OF MANAGEMENT TRAINING



In [2]:
dataset=pd.read_csv("management_training.csv")

In [3]:
print(dataset.isnull().sum())
print("shape: " + str(dataset.shape))

departament_id           0
intervention             0
engagement_score         0
tenure                   0
n_of_reports             0
gender                   0
role                     0
last_engagement_score    0
department_score         0
department_size          0
dtype: int64
shape: (10391, 10)


In [4]:
muestras_por_valor = dataset['intervention'].value_counts()
print("Samples with treatment:", muestras_por_valor[1])
print("Samples with treatment:", muestras_por_valor[0])

Samples with treatment: 5611
Samples with treatment: 4780


In [5]:
dataset.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843


Seeking if there is any shadow discrete variable:

In [6]:
num_unique_values = dataset.nunique()
# Iterate over the columns and print the number of unique values
for column, num_values in num_unique_values.items():
    print(f"{column}: {num_values}")

departament_id: 76
intervention: 2
engagement_score: 10391
tenure: 7
n_of_reports: 8
gender: 2
role: 5
last_engagement_score: 10391
department_score: 76
department_size: 72


## PLOTS TO IDENTIFY CONFOUNDERS

It is possible to observe that density of points with treatment is more near to the right plot part.

In [7]:
(ggplot(dataset, aes(x='last_engagement_score', y='engagement_score', color=as_discrete('intervention'))) +
geom_point() +
ggmarginal("t", layer=geom_density()) +
ggtitle("engagement_score by last_engagement_score and intervention") +
xlab('last_engagement_score') +
ylab('engagement_score') +
theme_bw())

An increasing trend may be observed. As the tenure increases, so does the engagement score in both the control and treated groups.

In [8]:
ggplot(dataset, aes(y='engagement_score', fill=as_discrete('intervention'))) + \
    geom_boxplot(aes(x='tenure'), notched=True)+ \
ggtitle("engagement_score by tenure and intervention")

It is observed that the most senior managers, with tenure = 6 or 7, receive more treatment. Furthermore, managers with less experience receive much less treatment.

In [9]:
ggplot(dataset,aes(x='tenure', fill=as_discrete('intervention'))) + geom_histogram(alpha=0.5, position='identity') + \
ggtitle("Tenure histogram by intervention")

In [10]:
ggplot(dataset,aes(x='department_score', fill=as_discrete('intervention'))) + geom_histogram(alpha=0.5, position='identity') + \
ggtitle("Department_score histogram by intervention")

It can be seen in the following graph that in the case of role 2, the people who receive the treatment are almost twice as many as the people who do not receive it. There is a possibility that because you have role 2, you are more likely to receive the treatment.

In [11]:
ggplot(dataset,aes(x='role', fill=as_discrete('intervention'))) + geom_histogram(alpha=0.5, position='identity') + \
ggtitle("Role histogram by intervention")

As in chapter 5 of the book it is not specified whether gender 1 is a woman or a man, I will use gender 1 and 2. Gender 1 has a higher engagement_score than gender 2 regardless of whether the control or treatment group is observed. That is, regardless of the treatment, gender 1 will have a higher engagement_score.

In [12]:
ggplot(dataset, aes(y='engagement_score', fill=as_discrete('intervention'))) + \
    geom_boxplot(aes(x='gender'), notched=True)+ \
    ggtitle("engagement_score by gender and intervention")

In the following graph you can see that n_of_reports causes bias since without treatment, n_reports = 5 already has a higher average engagement_score. This means that when the treatment is applied to cases with n_reports=5, we cannot attribute a higher engagement_score solely to the treatment.

In [13]:
ggplot(dataset, aes(y='engagement_score', fill=as_discrete('intervention'))) + \
    geom_boxplot(aes(x='n_of_reports'), notched=True)+ \
    ggtitle("engagement_score by n_of_reports and intervention")

In [14]:
ggplot(dataset,aes(x='n_of_reports', fill=as_discrete('intervention'))) + geom_histogram(alpha=0.5, position='identity') + \
ggtitle("n_of_reports histogram by intervention")

Larger departments tend to receive the treatment more than smaller departments.

In [15]:
ggplot(dataset,aes(x='department_size', fill=as_discrete('intervention'))) + geom_histogram(alpha=0.5, position='identity') + \
ggtitle("Department_size histogram by intervention")

In [16]:
ggplot(dataset, aes(x='department_size', y='engagement_score', color=as_discrete('intervention'))) + \
geom_point() + \
ggmarginal("t", layer=geom_density())

## FWL
Although it only seems to be interesting to apply FWL with tenure as variable X, I am going to compare FWL with tenure, last_engagement_score,department_score, C(n_of_reports),C(gender), C(role) as X (**case 1**) versus linear regression with the samevariables and treatment "intervention" (**case 3**).


## CASE 1

FWL step by step considering tenure, last_engagement_score, department_score, C(n_of_reports),C(gender),C(role) confounders X as Matheus Facure does in the Causal Inference book at Chapter 5, page 143-143.

As the calculations on steps 1 and 2 are the same but with different parameters it is posssible to create a function to avoid repeat code.

### **1.1.Debiasing Step**:
As Josh Tankard comments in this [article](https://medium.com/@jcatankard_76170/frisch-waugh-lovell-theorem-causal-inference-5add0d854019) although the treatment is binary we CAN NOT use any model of classification to this step. That is why regardless the binary treatment it is used a linear regression model instead of a logistic regression model.

In [22]:
def residual_around_0_for_continuous_objective(y:str, x:list, df):
    formula=f"{y}~{'+'.join(x)}"
    linear_regression_model = smf.ols(formula, data=df).fit()

    # the mean of the original y is added to center te y_residual around 0
    y_residual= linear_regression_model.resid + df[y].mean()

    return y_residual


In [23]:
variables_x=['tenure', 'last_engagement_score', 'department_score',  'C(n_of_reports)', 'C(gender)', 'C(role)']

In [24]:
residual_t=residual_around_0_for_continuous_objective("intervention", variables_x, dataset)

In [25]:
df_aux=dataset.assign(residual_t=residual_t)

In [26]:
(ggplot(df_aux, aes(x='residual_t', y='engagement_score')) +
geom_point(aes(color=as_discrete('intervention'))) + \
 ggtitle("engagement_score by residual of treatment got on case 1") +
theme_bw())

### **1.2.Denoising step**

In [27]:
residual_y=residual_around_0_for_continuous_objective("engagement_score",variables_x,dataset)

In [28]:
df_aux=df_aux.assign(residual_y=residual_y)

### **1.3.Final outcome**

In [29]:
fwl_last_step_model = smf.ols("residual_y ~ residual_t", data=df_aux).fit()
print(fwl_last_step_model.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1431      0.013    -11.386      0.000      -0.168      -0.118
residual_t     0.2651      0.017     15.299      0.000       0.231       0.299


In [30]:
(ggplot(df_aux, aes(x='residual_t', y='residual_y')) +
geom_point(aes(color=as_discrete('intervention'))) + ggtitle("residual engagement score by residual intervention Case 1") +
geom_smooth( method="lm", se=False) +
theme_bw())

## CASE 2

In this case, the same variables from case 1 are consider parto of X variables + department_size

### 2.1.Debiasing step

In [31]:
variables_x = ['tenure', 'last_engagement_score', 'department_score',  'C(n_of_reports)', 'C(gender)', 'C(role)'] + ['department_size']
residual_t = residual_around_0_for_continuous_objective("intervention", variables_x, dataset)
df_aux = dataset.assign(residual_t=residual_t)
(ggplot(df_aux, aes(x='residual_t', y='engagement_score')) +
 geom_point(aes(color=as_discrete('intervention'))) + \
 ggtitle("engagement_score by residual of treatment got on case 2") +
 theme_bw())

### 2.2.Denoising step

In [32]:
residual_y = residual_around_0_for_continuous_objective("engagement_score", variables_x, dataset)
df_aux = df_aux.assign(residual_y=residual_y)

### **2.3.Final outcome**

In [33]:
fwl_last_step_model = smf.ols("residual_y ~ residual_t", data=df_aux).fit()
print(fwl_last_step_model.summary().tables[1])
(ggplot(df_aux, aes(x='residual_t', y='residual_y')) +
 geom_point(aes(color=as_discrete('intervention'))) + ggtitle("residual engagement score by residual intervention Case 2") +
 geom_smooth(method="lm", se=False) +
 theme_bw())

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1447      0.013    -11.525      0.000      -0.169      -0.120
residual_t     0.2680      0.017     15.483      0.000       0.234       0.302


## QUESTION 3: LINEAR REGRESSION AS POTENTIAL OUTCOME IMPUTATION TECHNIQUE

To evaluate if the control population in the dataset yields good approximation to potentials outcomes Y0 and Y1 conditioning on variables X, it is possible to use linear regression as an outcome imputation technique.

First, it is necessary to verify if using the current population it is possible to obtain good approximations of the potentials outcome. Accuracy metric will be used to evaluate.

In [34]:
# getting the population without treatment
no_treated_dataset=dataset[dataset['intervention'] == 0]

# getting only the objective column and variables X
no_treated_dataset=no_treated_dataset[['tenure', 'last_engagement_score', 'department_score',  'n_of_reports', 'gender', 'role', 'engagement_score']]

# splitting the dataset into train and test
train_data, test_data = train_test_split(no_treated_dataset, test_size=0.2, random_state=42)

# creating and training a linear regression model
variables_x = ['tenure', 'last_engagement_score', 'department_score', 'C(n_of_reports)', 'C(gender)', 'C(role)']
model_without_treatment=smf.ols(f"engagement_score ~  {'+'.join(variables_x)}", data=train_data).fit()

# taking out the objective variable from the data test
y_test=test_data['engagement_score']
x_test=test_data.drop(['engagement_score'], axis=1)
predictions = model_without_treatment.predict(x_test)

# evaluating the accuracy to predict with MSE
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.748431666416042


Now, an evaluation of the capacity of potential outcome imputation with the population with treatment:

In [35]:
# getting the population with treatment
treated_dataset=dataset[dataset['intervention'] == 1]

# getting only the objective column and variables X
treated_dataset=treated_dataset[['tenure', 'last_engagement_score', 'department_score', 'n_of_reports', 'gender', 'role', 'engagement_score']]

# splitting the dataset into train and test
train_data, test_data = train_test_split(treated_dataset, test_size=0.2, random_state=42)

# creating and training a linear regression model
variables_x = ['tenure', 'last_engagement_score', 'department_score',  'C(n_of_reports)', 'C(gender)', 'C(role)']
model_with_treatment=smf.ols(f"engagement_score ~  {'+'.join(variables_x)}", data=train_data).fit()

# taking out the objective variable from the data test
y_test=test_data['engagement_score']
x_test=test_data.drop(['engagement_score'], axis=1)
predictions = model_with_treatment.predict(x_test)

# evaluating the accuracy to predict with MSE
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.7060570882522146


As mean squared error in both models, to predict potential outcomes Y0 and Y1, is above 70%, it is possible to calculate the ATE using linear regression as potencial outcome imputations.

In [36]:
# predicting potential outcome Y1 from samples without treatment
x_y0=no_treated_dataset.drop(['engagement_score'], axis=1)
y1_outcome = model_with_treatment.predict(x_y0)

# difference of engagement score over samples without treatment
effect_over_not_treated = y1_outcome - no_treated_dataset['engagement_score']

# predicting potential outcome Y0 from samples with treatment
x_y1=treated_dataset.drop(['engagement_score'], axis=1)
y0_outcome = model_without_treatment.predict(x_y1)

# difference of engagement score over samples without treatment
effect_over_treated = treated_dataset['engagement_score'] - y0_outcome

# calculation Average Treatment Effect
ATE = (effect_over_treated.sum() + effect_over_not_treated.sum())/dataset.shape[0]
print('ATE:',ATE)

ATE: 0.2655420211671395


## QUESTION 4
It is going to be analyze if it is possible to obtain an ATE with les SE dropping variable tenure.

In [37]:
variables_x = ['C(intervention)','last_engagement_score', 'department_score',  'C(n_of_reports)', 'C(gender)', 'C(role)']
fwl_last_step_model = smf.ols(f"engagement_score ~ {'+'.join(variables_x)}", data=dataset).fit()
print(fwl_last_step_model.summary().tables[1])

                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.1229      0.047      2.638      0.008       0.032       0.214
C(intervention)[T.1]      0.4212      0.019     22.374      0.000       0.384       0.458
C(n_of_reports)[T.2]      0.0019      0.039      0.049      0.961      -0.075       0.079
C(n_of_reports)[T.3]     -0.1507      0.095     -1.592      0.111      -0.336       0.035
C(n_of_reports)[T.4]     -0.0665      0.034     -1.945      0.052      -0.134       0.001
C(n_of_reports)[T.5]      0.4824      0.056      8.642      0.000       0.373       0.592
C(n_of_reports)[T.6]     -0.0093      0.153     -0.061      0.952      -0.309       0.291
C(n_of_reports)[T.7]     -0.1499      0.148     -1.011      0.312      -0.441       0.141
C(n_of_reports)[T.8]     -0.0272      0.037     -0.736      0.462      -0.100       0.045
C(gender)[

If the SE of intervention is compared to the previous one(0.17) it is possible to see that it has increased.